In [2]:
import os
import pandas as pd
from torch.utils.data import Dataset
import numpy as np

In [3]:
def process_data(y_dir, x_dir, gages, gages1, flow, seq_length, input_size):
    X = np.zeros((flow.shape[0], seq_length, input_size))
    for i, basin in gages.iterrows():
        df2 = pd.read_csv(os.path.join(y_dir, basin['STAID'] + '_Q_Month.txt'),
                         skiprows=38, delimiter=';', parse_dates=[0], dayfirst=False, encoding='cp1252')
        # 判断是否有至少一行 'Original' column 中的值为 -999.000
        if -999.000 in df2[' Original'].values:
            # 使用 'Calculated' 数据替换 'Original' 数据中值为 -999.000 的行
            df2.loc[df2[' Original'] == -999.000, ' Original'] = df2.loc[df2[' Original'] == -999.000, ' Calculated']
        df2 = df2.drop(columns=[' Calculated'])
        # 将时间列转换为日期时间格式
        df2['YYYY-MM-DD'] = pd.to_datetime(df2['YYYY-MM-DD'])
        df2['Y'] = df2['YYYY-MM-DD'].dt.year
        df2['M'] = df2['YYYY-MM-DD'].dt.month
        # 筛选出时间在1951年之后的行
        df2 = df2[(df2['YYYY-MM-DD'].dt.year >= 1951) & (df2['YYYY-MM-DD'].dt.year <= 2020)].reset_index(drop=True)
        beg_year = df2['YYYY-MM-DD'].dt.year.iloc[0]
        beg_month = df2['YYYY-MM-DD'].dt.month.iloc[0]
        df1 = pd.read_csv(os.path.join(x_dir, basin['STAID'] + '.csv'), parse_dates=[0])
        # 将时间列转换为日期时间格式
        df1['time'] = pd.to_datetime(df1['time'])
        if (df2['YYYY-MM-DD'].dt.year.iloc[0] == 1951) and (df2['YYYY-MM-DD'].dt.month.iloc[0] == 1):
            # 获取时间列
            time_column = df2['YYYY-MM-DD']
            # 使用isin方法过滤第二个CSV文件的数据
            df1 = df1[df1['time'].isin(time_column)]
        else:
            df1['Y'] = df1['time'].dt.year
            df1['M'] = df1['time'].dt.month
            # 根据逆推公式获取预测变量的开始年份和月份
            pred_var_beg_year = beg_year - abs(seq_length - beg_month) // 12 - 1
            pred_var_beg_month = 13 - abs(seq_length - beg_month) % 12
            # 筛选出时间在径流数据起始时间之后的行
            df1 = df1[
                ((df1['Y'] > pred_var_beg_year) | ((df1['Y'] == pred_var_beg_year) & (df1['M'] >= pred_var_beg_month)))]
        df1 = df1.iloc[:, 1:input_size+1].reset_index(drop=True)
        basin_idx = flow['basin'] == i
        delta = flow['delta'][basin_idx]
        idx_rep = np.repeat(delta, seq_length)
        if basin['STAID'] in gages1['STAID'].values:
            idx_sub = np.tile(np.arange(0, seq_length), delta.shape[0])
            dx = idx_rep + idx_sub
        else:
            idx_sub = np.tile(np.arange(0, seq_length)[::-1], delta.shape[0])
            dx = idx_rep - idx_sub  
        X[basin_idx] = np.array(df1.iloc[dx]).reshape(X[basin_idx].shape)
    return X

In [4]:
class Dynamic(Dataset):

    # load the dataset
    def __init__(self, gages_dir, gages_dir1, xd_dir, xg_dir, y_dir, seq_length, input_size_dyn, input_size_glo, output_size):
        flow = pd.DataFrame()
        gages = pd.read_excel(gages_dir, usecols=['STAID'], dtype=str)  # 获取站点编号
        gages1 = pd.read_excel(gages_dir1, usecols=['STAID'], dtype=str)
        gages['num_months'] = -1

        for i, basin in gages.iterrows():
            df = pd.read_csv(os.path.join(y_dir, basin['STAID'] + '_Q_Month.txt'),
                             skiprows=38, delimiter=';', parse_dates=[0], dayfirst=False, encoding='cp1252')
            # 判断是否有至少一行 'Original' column 中的值为 -999.000
            if -999.000 in df[' Calculated'].values:
                # 使用 'Original' 数据替换 'Calculated' 数据中值为 -999.000 的行
                df.loc[df[' Calculated'] == -999.000, ' Calculated'] = df.loc[df[' Calculated'] == -999.000, ' Original']
            df = df.drop(columns=[' Original'])
            # Set -999.000 values to NaN
            df[' Calculated'] = df[' Calculated'].replace(-999.000, np.nan)
            # 将时间列转换为日期时间格式
            df['YYYY-MM-DD'] = pd.to_datetime(df['YYYY-MM-DD'])
            df['Y'] = df['YYYY-MM-DD'].dt.year
            df['M'] = df['YYYY-MM-DD'].dt.month
            # 筛选出时间在1951年之后的行
            df = df[(df['YYYY-MM-DD'].dt.year >= 1951) & (df['YYYY-MM-DD'].dt.year <= 2020)].reset_index(drop=True)
            beg_year = df['YYYY-MM-DD'].dt.year.iloc[0]
            beg_month = df['YYYY-MM-DD'].dt.month.iloc[0]
            # # 对前 80% 行进行线性插值
            # num_rows_to_interpolate = np.rint(0.8 * (df.shape[0])).astype('int')
            # df.iloc[:num_rows_to_interpolate] = df.iloc[:num_rows_to_interpolate].interpolate(method='linear')
            if (df['YYYY-MM-DD'].dt.year.iloc[0] == 1951) and (df['YYYY-MM-DD'].dt.month.iloc[0] == 1):
                flow_beg = np.searchsorted((df['Y'] == beg_year + (beg_month + seq_length - 2) // 12) &
                                           (df['M'] >= (beg_month + seq_length - 2) % 12 + 1) |
                                           (df['Y'] >= beg_year + (beg_month + seq_length - 2) // 12 + 1), True)
            else:
                flow_beg = 0

            if flow_beg < df.shape[0]:
                df['delta'] = (df['Y'] - beg_year) * 12 + df['M'] - beg_month
                flow = pd.concat([flow, df.iloc[flow_beg:]], axis=0, ignore_index=True)
                gages.loc[i, 'num_months'] = df.iloc[flow_beg:].shape[0]

        gages = gages[gages['num_months'] != -1]
        gages['t'] = np.cumsum(gages['num_months'])
        gages['s'] = gages['t'].shift(1, fill_value=0)
        flow['basin'] = np.repeat(np.arange(0, gages.shape[0], dtype=int), gages['t'] - gages['s'])
        Xg = process_data(y_dir, xg_dir, gages, gages1, flow, seq_length , input_size_glo)
        Xd = process_data(y_dir, xd_dir, gages, gages1, flow, seq_length , input_size_dyn)
        y = flow[' Calculated'].values.reshape((flow.shape[0], output_size))
        basin = flow['basin'].values
        date_integers = flow['YYYY-MM-DD'].map(lambda date: date.toordinal())
        
        
#         # 找到非空 y 值的索引
#         non_empty_indices = np.where(~np.isnan(y.flatten()))[0]
         
#         # 提取非空 y 值对应的 x 值
#         Xg = Xg[non_empty_indices]
#         Xd = Xd[non_empty_indices]
#         y = y[non_empty_indices]
#         basin = basin[non_empty_indices]
        # ensure inputs and target has the right dtype
        self.gages = gages
        self.Xg = Xg.astype('float32')
        self.Xd = Xd.astype('float32')
        self.y = y.astype('float32')
        self.basin = basin.astype('int32')
        self.time = date_integers.values.astype('int32')

        # prevent subclass from calling Dynamic.normalize
        self.norm = None
        if isinstance(self, Dynamic):
            self.normalize()

    # number of rows in the dataset
    def __len__(self):
        return [len(self.Xd),len(self.Xg)]

    # get a row at an index
    def __getitem__(self, idx):
        return [self.Xd[idx], self.Xg[idx], self.y[idx], self.basin[idx], self.time[idx]]

    # normalization
    def normalize(self, norm=None):
        # cannot normalize twice
        if self.norm is not None:
            return self.norm

        # scaler of Xd
        if norm is None:
            Xd_mean = self.Xd.reshape(-1, self.Xd.shape[-1]).mean(axis=0)
            Xd_std = self.Xd.reshape(-1, self.Xd.shape[-1]).std(axis=0)
        else:
            Xd_mean = norm[0]
            Xd_std = norm[1]
        # scaler of X
        if norm is None:
            Xg_mean = self.Xg.reshape(-1, self.Xg.shape[-1]).mean(axis=0)
            Xg_std = self.Xg.reshape(-1, self.Xg.shape[-1]).std(axis=0)
        else:
            Xg_mean = norm[2]
            Xg_std = norm[3]
        # scaler of y
        if norm is None:
            y_mean = 0
            # y_std = self.y.max(axis=0) - y_mean #有空值不可用
            y_std = np.nanmax(self.y, axis=0) - y_mean
        else:
            y_mean = norm[4]
            y_std = norm[5]

        # normalize X and y
        self.Xd = (self.Xd - Xd_mean) / Xd_std
        self.Xg = (self.Xg - Xg_mean) / Xg_std
        self.y = (self.y - y_mean) / y_std
        self.norm = (Xd_mean, Xd_std, Xg_mean, Xg_std, y_mean, y_std)
    
        return self.norm
    

class WithStatic(Dynamic):
    # load the dataset
    def __init__(self, gages_dir, gages_dir1, S_dir, xd_dir,  xg_dir, y_dir, seq_length, 
                 input_size_dyn, input_size_glo, input_size_sta, output_size):
        super().__init__( gages_dir, gages_dir1, xd_dir, xg_dir, y_dir, seq_length, 
                         input_size_dyn, input_size_glo, output_size)
        df_sta = pd.read_csv(S_dir, dtype={'STAID': str})
        S = pd.merge(self.gages, df_sta, how='left', on='STAID').iloc[:, -input_size_sta:]
        self.S = S.values[self.basin].astype('float32')
        # normalization static
        self.normalize()
    def __getitem__(self, idx):
        return [(self.S[idx], self.Xd[idx], self.Xg[idx]), self.y[idx], self.basin[idx], self.time[idx]]
    def normalize(self, norm=None):
        # cannot normalize twice
        if self.norm is not None and len(self.norm) == 8:
            return self.norm
        if self.norm is None:
            if norm is None:
                super().normalize()
            else:
                super().normalize(norm=norm[2:])
        if hasattr(self, 'S'):
            if norm is None:
                S_mean = self.S.mean(axis=0)
                S_std = self.S.std(axis=0)
            else:
                S_mean = norm[0]
                S_std = norm[1]
            self.S = (self.S - S_mean) / S_std
            self.norm = (S_mean, S_std, *self.norm)
        return self.norm


In [5]:
class WithStatic(Dynamic):
    # load the dataset
    def __init__(self, gages_dir, gages_dir1, S_dir, xd_dir,  xg_dir, y_dir, seq_length, 
                 input_size_dyn, input_size_glo, input_size_sta, output_size):
        super().__init__( gages_dir, gages_dir1, xd_dir, xg_dir, y_dir, seq_length, 
                         input_size_dyn, input_size_glo, output_size)
        df_sta = pd.read_csv(S_dir, dtype={'STAID': str})
        S = pd.merge(self.gages, df_sta, how='left', on='STAID').iloc[:, -input_size_sta:]
        self.S = S.values[self.basin].astype('float32')
        # normalization static
        self.normalize()
    def __getitem__(self, idx):
        return [(self.S[idx], self.Xd[idx], self.Xg[idx]), self.y[idx], self.basin[idx], self.time[idx]]
    def normalize(self, norm=None):
        # cannot normalize twice
        if self.norm is not None and len(self.norm) == 8:
            return self.norm
        if self.norm is None:
            if norm is None:
                super().normalize()
            else:
                super().normalize(norm=norm[2:])
        if hasattr(self, 'S'):
            if norm is None:
                S_mean = self.S.mean(axis=0)
                S_std = self.S.std(axis=0)
            else:
                S_mean = norm[0]
                S_std = norm[1]
            self.S = (self.S - S_mean) / S_std
            self.norm = (S_mean, S_std, *self.norm)
        return self.norm


In [7]:
S_dir = 'data/balstm/grdc_attributes1.csv'
gages_dir = 'data/balstm/basin_list.xlsx'
gages_dir1 = 'data/balstm/basin_def.xlsx'
y_dir = 'data/balstm/runoff_data/txt'
xd_dir = 'data/balstm/forcing_t'
xg_dir = 'data/balstm/global_data/output'
seq_length = 12
input_size_dyn = 13
input_size_glo = 106
output_size = 1
input_size_sta = 195
dataset = WithStatic(gages_dir, gages_dir1, S_dir, xd_dir,  xg_dir, y_dir, seq_length, 
                 input_size_dyn, input_size_glo, input_size_sta, output_size)

In [19]:
print(dataset.S.shape)
print(dataset.Xd.shape)
print(dataset.Xg.shape)
print(dataset.y.shape)
print(dataset.basin.shape)
print(dataset.time.shape)


(120281, 195)
(120281, 12, 13)
(120281, 12, 106)
(120281, 1)
(120281,)
(120281,)
[712557 712588 712619 ... 726741 726772 726802]


In [9]:
# y_flat = dataset.y.flatten()
# is_nan = np.isnan(y_flat)

# # Count the number of NaN values by summing the boolean array
# num_nan = np.sum(is_nan)
# num = np.sum(y_flat)
# print(num_nan)
# print(num)
# print(dataset.y)
# Create a boolean array where True represents a NaN value
is_nan = np.isnan(dataset.Xg)

# Count the number of NaN values by summing the boolean array
num_nan = np.sum(is_nan)

# Calculate the proportion of NaN values
proportion_nan = num_nan / np.size(dataset.Xd)

print("Number of NaN values:", num_nan)
print("Proportion of NaN values:", proportion_nan)

Number of NaN values: 0
Proportion of NaN values: 0.0


In [22]:
from torch.utils.data import Subset, DataLoader


def gen_loader(dataset, splits, **loader_args):
    for train_idx, test_idx in splits:
        train_loader = DataLoader(Subset(dataset, train_idx),
                                  **loader_args, shuffle=True)
        test_loader = DataLoader(Subset(dataset, test_idx),
                                 **loader_args, shuffle=False)
        yield train_loader, test_loader


class BaseSplit:
    def __init__(self, batch_size=32, num_workers=0, pin_memory=False) -> None:
        super().__init__()
        self.loader_args = {
            'batch_size': batch_size,
            'num_workers': num_workers,
            'pin_memory': pin_memory
        }

    def __iter__(self):
        return gen_loader(self.dataset, self.splits, **self.loader_args)
    
class TrainTestSplit(BaseSplit):
    def __init__(self, dataset, batch_size=32, num_workers=0,
                 pin_memory=False, test_size=0.2) -> None:
        super().__init__(batch_size, num_workers, pin_memory)
        # 获取每个样本的流域标签
        basins = dataset.basin
        # 唯一流域标签
        unique_basins = np.unique(basins)
        # 初始化训练和测试索引列表
        train_idx, test_idx = [], []
        # 对每个流域执行按比例划分
        for basin in unique_basins:
            # 当前流域的所有索引
            idxs = np.where(basins == basin)[0]
            # 计算训练集大小
            train_size_int = int(len(idxs) * (1 - test_size))
            # 根据比例划分当前流域的数据
            # 这里按顺序划分，不随机
            basin_train_idx = idxs[:train_size_int]
            basin_test_idx = idxs[train_size_int:]
            # 添加到总的训练和测试索引列表
            train_idx.extend(basin_train_idx)
            test_idx.extend(basin_test_idx)
        # 转换为numpy数组
        train_idx = np.array(train_idx)
        test_idx = np.array(test_idx)
        # 保存划分结果
        self.dataset = dataset
        self.splits = [(train_idx, test_idx)]

In [23]:
import numpy as np
from sklearn.model_selection import train_test_split


# def multi_arange(starts, stops, steps=1):
#     lens = ((stops - starts) + steps - np.sign(steps)) // steps
#     if isinstance(steps, np.ndarray):
#         res = np.repeat(steps, lens)
#     else:
#         res = np.full(lens.sum(), steps)
#     ends = (lens - 1) * steps + starts
#     res[0] = starts[0]
#     res[lens[:-1].cumsum()] = starts[1:] - ends[:-1]
#     return res.cumsum()


# def train_test_stratify(labels, test_size):
#     bin_counts = np.bincount(labels)
#     counts = bin_counts[np.nonzero(bin_counts)[0]]
#     end = np.cumsum(counts)
#     test_counts = np.rint(counts * test_size).astype('int')
#     split = end - test_counts

#     test_idx = multi_arange(split, end).astype(int)
#     train_idx = np.setdiff1d(np.arange(len(labels)), test_idx)

#     return train_idx, test_idx


# class TrainTestSplit(BaseSplit):
#     def __init__(self, dataset, batch_size=32, num_workers=0,
#                  pin_memory=False, test_size=0.2) -> None:
#         super().__init__(batch_size, num_workers, pin_memory)

#         # Split dataset into train and test
#         if test_size:
#             split = train_test_stratify(dataset.basin, test_size)
#             # split = train_test_split(
#             #     np.arange(len(dataset.basin)),
#             #     test_size=test_size,
#             #     random_state=1,
#             #     shuffle=False,
#             #     stratify=dataset.basin)
#             # print(split)
#         else:
#             split = (np.arange(len(dataset)), np.array([]))

#         self.dataset = dataset
#         self.splits = [split]

class TrainTestSplit(BaseSplit):
    def __init__(self, dataset, batch_size=32, num_workers=0,
                 pin_memory=False, test_size=0.2) -> None:
        super().__init__(batch_size, num_workers, pin_memory)

        # 获取每个样本的流域标签
        basins = dataset.basin
        
        # 唯一流域标签
        unique_basins = np.unique(basins)
        
        # 初始化训练和测试索引列表
        train_idx, test_idx = [], []

        # 对每个流域执行按比例划分
        for basin in unique_basins:
            # 当前流域的所有索引
            idxs = np.where(basins == basin)[0]
            # 计算训练集大小
            train_size_int = int(len(idxs) * (1 - test_size))
            
            # 根据比例划分当前流域的数据
            # 这里按顺序划分，不随机
            basin_train_idx = idxs[:train_size_int]
            basin_test_idx = idxs[train_size_int:]
            
            # 添加到总的训练和测试索引列表
            train_idx.extend(basin_train_idx)
            test_idx.extend(basin_test_idx)

        # 转换为numpy数组
        train_idx = np.array(train_idx)
        test_idx = np.array(test_idx)

        # 保存划分结果
        self.dataset = dataset
        self.splits = [(train_idx, test_idx)]


In [29]:
datasplit = TrainTestSplit(dataset, batch_size=32, num_workers=0,
                 pin_memory=False, test_size=0.2)
print(datasplit.splits)

[(array([     0,      1,      2, ..., 120184, 120185, 120186]), array([   567,    568,    569, ..., 120278, 120279, 120280]))]


In [13]:
# for train_loader, test_loader in datasplit:
#     for batch_idx, (data, target, basin, time) in enumerate(test_loader):
#         print(basin)
import random
import torch
import numpy as np

def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [14]:
from typing import Tuple
import torch
import torch.nn as nn

class BALSTM(nn.Module):
    def __init__(self, input_size_sta, input_size_dyn, input_size_glo, hidden_size, output_size, num_layers, drop_prob=0.5):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # 使用SingleBALSTM作为子模块
        self.lstm = SingleBALSTM(input_size_sta, input_size_dyn, input_size_glo, hidden_size,
                                 batch_first=True, initial_forget_bias=0)

        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_size, output_size)
        self.act = nn.ReLU()

    def forward(self, data):
        x_s, x_d, x_g = data

        # 前向传播LSTM
        # out: tensor的形状为(batch_size, seq_length, hidden_size)
        out, _ = self.lstm(x_s, x_d, x_g)

        # 解码最后一个时间步的隐藏状态
        out = self.fc(self.dropout(out[:, -1, :]))

        # 使用特定激活函数
        out = self.act(out)
        return out


class SingleBALSTM(nn.Module):
    """实现基于Basin-Aware-LSTM (BA-LSTM)的模型
    Parameters
    ----------
    input_size_dyn : int
        动态特征的数量，这些特征在每个时间步传递给LSTM。
    input_size_sta : int
        静态特征的数量，这些特征用于调制输入门。
    input_size_glo : int
        全局特征的数量，这些特征在每个时间步传递给LSTM。
    hidden_size : int
        隐藏/记忆单元的数量。
    batch_first : bool, optional
        如果为True，期望批量输入的形状为[batch, seq, features]；否则，形状应为[seq, batch, features]，默认为True。
    initial_forget_bias : int, optional
        初始遗忘门偏置的值，默认为0。
    """

    def __init__(self,
                 input_size_sta: int,
                 input_size_dyn: int,
                 input_size_glo: int,
                 hidden_size: int,
                 batch_first: bool = True,
                 initial_forget_bias: int = 0):
        super().__init__()

        self.input_size_dyn = input_size_dyn
        self.input_size_sta = input_size_sta
        self.input_size_glo = input_size_glo
        self.hidden_size = hidden_size
        self.batch_first = batch_first
        self.initial_forget_bias = initial_forget_bias

        # 创建可学习参数的张量
        self.weight_ih = nn.Parameter(torch.FloatTensor(input_size_dyn, 3 * hidden_size))   # 划分为三个门(f, d, o)
        self.weight_hh = nn.Parameter(torch.FloatTensor(hidden_size, 3 * hidden_size))
        self.weight_sh = nn.Parameter(torch.FloatTensor(input_size_sta, 2*hidden_size))  # 划分为两个门（i1，i2）
        self.weight_gh = nn.Parameter(torch.FloatTensor(input_size_glo, hidden_size))   # 划分一个门（g）
        self.bias = nn.Parameter(torch.FloatTensor(3 * hidden_size))
        self.bias_s = nn.Parameter(torch.FloatTensor(2 * hidden_size))
        self.bias_g = nn.Parameter(torch.FloatTensor(hidden_size))

        # 初始化参数
        self.reset_parameters()

    def reset_parameters(self):
        """初始化LSTM的所有可学习参数"""
        nn.init.orthogonal_(self.weight_ih.data)
        nn.init.orthogonal_(self.weight_gh.data)
        nn.init.orthogonal_(self.weight_sh)

        weight_hh_data = torch.eye(self.hidden_size)
        weight_hh_data = weight_hh_data.repeat(1, 3)
        self.weight_hh.data = weight_hh_data

        nn.init.constant_(self.bias.data, val=0)
        nn.init.constant_(self.bias_s.data, val=0)

        if self.initial_forget_bias != 0:
            self.bias.data[:self.hidden_size] = self.initial_forget_bias

    def forward(self, x_s: torch.Tensor, x_d: torch.Tensor, x_g: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """[summary]
        Parameters
        ----------
        x_d : torch.Tensor
            包含动态特征序列的批量张量。形状必须与batch_first指定的格式匹配。
        x_s : torch.Tensor
            包含静态特征的批量张量。
        x_g : torch.Tensor
            包含全局特征的批量张量。
        Returns
        -------
        h_n : torch.Tensor
            批量中每个样本每个时间步的隐藏状态。
        c_n : torch.Tensor
            批量中每个样本每个时间步的记忆状态。
        """
        if self.batch_first:
            x_d = x_d.transpose(0, 1)
            x_g = x_g.transpose(0, 1)   # (seq_len, batch_size, feature_size)            

        seq_len, batch_size, _ = x_d.size()

        h_0 = x_d.data.new(batch_size, self.hidden_size).zero_()
        c_0 = x_d.data.new(batch_size, self.hidden_size).zero_()
        h_x = (h_0, c_0)

        # 用于临时存储所有中间隐藏/记忆状态的空列表
        h_n, c_n = [], []

        # 将偏置向量扩展到批量大小
        bias_batch = (self.bias.unsqueeze(0).expand(batch_size, *self.bias.size()))
        bias_g_batch = (self.bias_g.unsqueeze(0).expand(batch_size, *self.bias_g.size()))
        # 仅计算输入门一次，因为输入是静态的
        bias_s_batch = (self.bias_s.unsqueeze(0).expand(batch_size, *self.bias_s.size()))
        i = (torch.addmm(bias_s_batch, x_s, self.weight_sh))
        i1, i2 = i.chunk(2, 1)
        i1 = torch.sigmoid(i1)
        i2 = torch.sigmoid(i2)
        
        # 在输入序列上执行前向步骤
        for t in range(seq_len):
            h_0, c_0 = h_x

            # 计算门
            g = torch.addmm(bias_g_batch, x_g[t], self.weight_gh)
            gates = (torch.addmm(bias_batch, h_0, self.weight_hh) +
                     torch.mm(x_d[t], self.weight_ih))
            f, d, o_h = gates.chunk(3, 1)
            o = o_h + torch.mm(x_g[t], self.weight_gh)
            c_1 = torch.sigmoid(f) * c_0 + i1 * torch.tanh(d) + i2 * torch.tanh(g)
            h_1 = torch.sigmoid(o) * torch.tanh(c_1)

            # 将中间隐藏/记忆状态存储在列表中
            h_n.append(h_1)
            c_n.append(c_1)

            h_x = (h_1, c_1)

        h_n = torch.stack(h_n, 0)
        c_n = torch.stack(c_n, 0)

        if self.batch_first:
            h_n = h_n.transpose(0, 1)
            c_n = c_n.transpose(0, 1)

        return h_n, c_n

In [15]:
import torch
from abc import abstractmethod, ABCMeta
from numpy import inf
from utils.logger import EpochMetrics
from pathlib import Path
from hydra.utils import to_absolute_path
import os
from shutil import copyfile
import logging
logger = logging.getLogger('base-trainer')

class BaseTrainer(metaclass=ABCMeta):
    """
    Base class for all trainers
    """
    @ abstractmethod
    def __init__(self, model, criterion, metric_ftns, optimizer, n_gpu, epochs, log_step, round):

        # setup GPU device if available, move model into configured device
        self.device, device_ids = self._prepare_device(n_gpu)
        self.model = model.to(self.device)
        if len(device_ids) > 1:
            self.model = torch.nn.DataParallel(model, device_ids=device_ids)

        self.criterion = criterion
        self.metric_ftns = metric_ftns
        self.optimizer = optimizer

        self.epochs = epochs
        self.log_step = log_step

        # setup metric monitoring for monitoring model performance and saving best-checkpoint
        self.monitor = 'min loss/valid'

        metric_names = ['loss'] + [type(met).__name__ for met in self.metric_ftns]
        self.ep_metrics = EpochMetrics(metric_names, phases=('train', 'valid'), monitoring=self.monitor)
        self.result_dir = f'epoch-results-{round}.csv' if round else f'epoch-results.csv'

        self.checkpt_top_k = 3
        self.early_stop = float('inf')

        self.start_epoch = 1
        self.checkpt_dir = Path('balstm/').absolute()
        self.checkpt_dir.mkdir(exist_ok=True)

    @abstractmethod
    def _train_epoch(self, epoch):
        """
        Training logic for an epoch

        :param epoch: Current epoch number
        """
        raise NotImplementedError

    def train(self):
        """
        Full training logic
        """
        not_improved_count = 0
        for epoch in range(self.start_epoch, self.epochs + 1):
            result = self._train_epoch(epoch)
            self.ep_metrics.update(epoch, result)

            # print result metrics of this epoch
            max_line_width = max(len(line) for line in str(self.ep_metrics).splitlines())
            # divider ---
            print('-' * max_line_width)
            print(self.ep_metrics.latest().to_string(float_format=lambda x: f'{x:.4f}') + '\n')

            # check if model performance improved or not, for early stopping and topk saving
            is_best = False
            improved = self.ep_metrics.is_improved()
            if improved:
                not_improved_count = 0
                is_best = True
            else:
                not_improved_count += 1

            if not_improved_count > self.early_stop:
                print("Validation performance didn\'t improve for {} epochs. "
                            "Training stops.".format(self.early_stop))
                break

            using_topk_save = self.checkpt_top_k > 0
            self._save_checkpoint(epoch, save_best=is_best, save_latest=using_topk_save)

            # keep top-k checkpoints only, using monitoring metrics
            if using_topk_save:
                self.ep_metrics.keep_topk_checkpt(self.checkpt_dir, self.checkpt_top_k)

            self.ep_metrics.to_csv(self.result_dir)

            # divider ===
            print('=' * max_line_width)

        # Return metric score for hyperparameter optimization
        return self.ep_metrics.latest_metric()

    def _prepare_device(self, n_gpu_use):
        """
        setup GPU device if available, move model into configured device
        """
        n_gpu = torch.cuda.device_count()
        if n_gpu_use > 0 and n_gpu == 0:
            logger.warning("Warning: There\'s no GPU available on this machine,"
                           "training will be performed on CPU.")
            n_gpu_use = 0
        if n_gpu_use > n_gpu:
            logger.warning("Warning: The number of GPU\'s configured to use is {}, but only {} are available "
                           "on this machine.".format(n_gpu_use, n_gpu))
            n_gpu_use = n_gpu
        device = torch.device('cuda:0' if n_gpu_use > 0 else 'cpu')
        list_ids = list(range(n_gpu_use))
        print('Using device: {} | GPU count: {}'.format(device, n_gpu_use))
        return device, list_ids
    
    def _save_checkpoint(self, epoch, save_best=False, save_latest=True):
        """
        Saving checkpoints

        :param epoch: current epoch number
        :param log: logging information of the epoch
        :param save_best: if True, save a copy of current checkpoint file as 'model_best.pth'
        :param save_latest: if True, save a copy of current checkpoint file as 'model_latest.pth'
        """
        state = {
            'model': type(self.model).__name__,
            'epoch': epoch,
            'state_dict': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'epoch_metrics': self.ep_metrics,
        }
        abs_path = self.checkpt_dir / 'checkpoint-epoch{}.pth'.format(epoch)
        torch.save(state, abs_path)


        rel_path = (
            abs_path.relative_to(os.getcwd())  # 使用 os.getcwd() 替代 get_original_cwd()
            if abs_path.is_relative_to(os.getcwd())
            else abs_path
        )
        print(f"Model checkpoint saved at: \n    {rel_path}")
        if save_latest:
            latest_path = self.checkpt_dir / 'model_latest.pth'
            copyfile(abs_path, latest_path)
        if save_best:
            best_path = self.checkpt_dir / 'model_best.pth'
            copyfile(abs_path, best_path)
            rel_best_path = (best_path.relative_to(Path.cwd())
                             if best_path.is_relative_to(Path.cwd())
                             else best_path)
            print(f"Renewing best checkpoint: \n    ...\\{rel_best_path}")

In [16]:
import torch
from utils.utils import inf_loop
from evaluate import evaluate
from utils.logger import BatchMetrics



class Trainer(BaseTrainer):
    """
    Trainer class
    """

    def __init__(self, model, criterion, metric_ftns, optimizer, n_gpu, epochs, log_step, data_loader,
                 valid_data_loader=None, lr_scheduler=None, len_epoch=None, round=0):
        super().__init__(model, criterion, metric_ftns, optimizer, n_gpu, epochs, log_step, round)
        self.data_loader = data_loader
        if len_epoch is None:
            # epoch-based training
            self.len_epoch = len(self.data_loader)
        else:
            # iteration-based training
            self.data_loader = inf_loop(data_loader)
            self.len_epoch = len_epoch
        self.valid_data_loader = valid_data_loader
        self.lr_scheduler = lr_scheduler

        self.train_metrics = BatchMetrics('loss',
                                          *[type(m).__name__ for m in self.metric_ftns],
                                          postfix='/train')
        self.valid_metrics = BatchMetrics('loss',
                                          *[type(m).__name__ for m in self.metric_ftns],
                                          postfix='/valid')
    def _train_epoch(self, epoch):
        """
        Training logic for an epoch

        :param epoch: Integer, current training epoch.
        :return: A log that contains average loss and metric in this epoch.
        """
        self.model.train()
        self.train_metrics.reset()
        outputs = []
        targets = []
        labels = []
        for batch_idx, (data, target, basin, time) in enumerate(self.data_loader):
            target = target.to(self.device)
            if isinstance(data, torch.Tensor):
                data = data.to(self.device)
            else:
                data = tuple(i.to(self.device) for i in data)

            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.criterion(output, target, basin)
            loss.backward()
            self.optimizer.step()

            # step = (epoch - 1) * self.len_epoch + batch_idx
            self.train_metrics.update('loss', loss.item())
            outputs.append(output)
            targets.append(target)
            labels.append(basin)
                       

            if batch_idx % self.log_step == 0:
                print('Train Epoch: {} {} Loss: {:.6f}'.format(
                    epoch,
                    self._progress(batch_idx),
                    loss.item()))

            if batch_idx == self.len_epoch:
                break
        
        output = torch.cat(outputs)
        target = torch.cat(targets)
        basin = torch.cat(labels)    
        for metric_ftn in self.metric_ftns:
            metric_value = metric_ftn(output, target, basin).cpu().detach().numpy()
            self.train_metrics.update(type(metric_ftn).__name__, metric_value)            
        
        
        # 添加训练期度量指标
        # metrics1 = evaluate(self.model, self.data_loader, *self.metric_ftns)
        # for metrict_ftn, metric1 in zip(self.metric_ftns, metrics1):
        #     self.train_metrics.update(type(metrict_ftn).__name__, metric1)

        log = self.train_metrics.result()
        

        if (self.valid_data_loader is not None and
                len(self.valid_data_loader) > 0):
            val_log = self._valid_epoch(epoch)
            log.update(**val_log)

        if self.lr_scheduler is not None:
            self.lr_scheduler.step()
        
        # # 打印整个训练周期的结果度量
        # for k, v in log.items():
        #     print(f"Epoch: {epoch}, Metric: {k}, Value: {v}")

        return log

    def _valid_epoch(self, epoch):
        """
        Validate after training an epoch

        :param epoch: Integer, current training epoch.
        :return: A log that contains information about validation
        """
        self.valid_metrics.reset()
        metrics = evaluate(self.model, self.valid_data_loader, [self.criterion, *self.metric_ftns])
        self.valid_metrics.update('loss', metrics[0])
        for metrict_ftn, metric in zip(self.metric_ftns, metrics[1:]):
            self.valid_metrics.update(type(metrict_ftn).__name__, metric)

        # add histogram of model parameters to the tensorboard
        # for name, p in self.model.named_parameters():
        #     self.writer.add_histogram(name, p, bins='auto')
        return self.valid_metrics.result()

    def _progress(self, batch_idx):
        base = '[{}/{} ({:.0f}%)]'
        try:
            # epoch-based training
            total = len(self.data_loader.dataset)
            current = batch_idx * self.data_loader.batch_size
        except AttributeError:
            # iteration-based training
            total = self.len_epoch
            current = batch_idx
        return base.format(current, total, 100.0 * current / total)


In [17]:
import numpy as np

In [18]:
from hydra.utils import instantiate
from trainer.loss import NMSELoss
from trainer.metric import *
import importlib

n_gpu = 1
epochs = 100
log_step = 1000
milestones = [700]
# 定义衰减比例
gamma = 0.5
learning_rate = 0.0001
n, accumulation = 0, 0
for train_loader, test_loader in datasplit:
    # build model and print its architecture
    model = BALSTM(input_size_sta, input_size_dyn, input_size_glo, hidden_size = 64,
                    output_size = 1, num_layers = 2, drop_prob = 0.4)
    trainable_params = filter(lambda p: p.requires_grad, model.parameters())
    criterion = NMSELoss(dataset)
   # 定义度量函数的完全限定名
    metrics_fqns = [
        "trainer.metric.NSE",
        "trainer.metric.MeanNSE",
        "trainer.metric.MedianNSE",
    ]
    # 实例化度量函数
    metrics = []
    for fqn in metrics_fqns:
        module_name, class_name = fqn.rsplit(".", 1)
        MetricClass = getattr(importlib.import_module(module_name), class_name)
        metrics.append(MetricClass())
    metrics = [met for met in metrics]
    # build optimizer and learning rate scheduler
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)
    # train and accumulate metric
    trainer = Trainer(model, criterion, metrics, optimizer, n_gpu, epochs, log_step, 
                      data_loader=train_loader, valid_data_loader=test_loader, 
                      lr_scheduler=lr_scheduler, round=n)
    accumulation += trainer.train()
    n += 1
print(accumulation / n)



Using device: cuda:0 | GPU count: 1
Train Epoch: 1 [0/96155 (0%)] Loss: 36.053913
Train Epoch: 1 [32000/96155 (33%)] Loss: 1.790355
Train Epoch: 1 [64000/96155 (67%)] Loss: 2.193565
Train Epoch: 1 [96000/96155 (100%)] Loss: 1.947436
-----------------------------------------------------------------------
           loss            NSE          MeanNSE         MedianNSE        
          train  valid   train   valid    train   valid     train   valid
epoch-1 15.3873 1.6053 -0.5551 -0.3749 -25.9265 -3.1005   -4.5043 -1.3173

Model checkpoint saved at: 
    balstm\checkpoint-epoch1.pth
Renewing best checkpoint: 
    ...\balstm\model_best.pth
Train Epoch: 2 [0/96155 (0%)] Loss: 1.453934
Train Epoch: 2 [32000/96155 (33%)] Loss: 2.353580
Train Epoch: 2 [64000/96155 (67%)] Loss: 1.218425


In [ ]:
# for input, target, label in test_loader:
#     print(len(label))
#     # print(target)

In [31]:
import torch
for train_loader, test_loader in datasplit:
    for batch_idx, (data, target, basin, time) in enumerate(train_loader):
        for tensor in data:
            is_nan = torch.isnan(tensor)
            if torch.any(is_nan):
                print("Found a NaN value!")
# Calculate the proportion of NaN values
#         proportion_nan = num_nan / np.size(dataset.y)

#         print("Number of NaN values:", num_nan)
#         print("Proportion of NaN values:", proportion_nan)

In [ ]:
import datetime
import pandas as pd
from trainer.metric import *

# Assuming you have the normalize function defined
def inverse_normalize(data, mean, std):
    return data * std + mean

_, _, _, _, _, _, y_mean, y_std = dataset.norm
pth = torch.load('balstm/model_latest.pth')
test_loader = next(iter(datasplit))[1]
# load trained weights
if n_gpu > 1:
    model = torch.nn.DataParallel(model)
model.load_state_dict(pth['state_dict'])
# prepare model for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

# simulate
with torch.no_grad():
    outputs = []
    targets = []
    labels = []
    times = []
    for input, target, label, time in test_loader:
        
        target = target.to(device)
        if isinstance(input, torch.Tensor):
            input = input.to(device)
        else:
            input = tuple(i.to(device) for i in input)
        outputs.append(model(input))
        targets.append(target)
        labels.append(label)
        times.append(time)
    
    # concatenate in dim 0
    y_pred = torch.cat(outputs)
    y_true = torch.cat(targets)
    basin = torch.cat(labels)
    Time = torch.cat(times)

mask = ~torch.isnan(y_true)  # Create a mask to exclude NaN values
y_pred = y_pred[mask]
y_true = y_true[mask]
mask = mask.flatten().cpu()
basin = basin[mask]
Time = Time[mask]
# 反归一化 y_true and y_pred
y_true_denorm = inverse_normalize(y_true.cpu().numpy(), y_mean, y_std)
y_pred_denorm = inverse_normalize(y_pred.cpu().numpy(), y_mean, y_std)
# 创建一个向量化函数
fromordinal_vectorized = np.vectorize(lambda ordinal: datetime.datetime.fromordinal(ordinal).date())
# 使用向量化函数转换时间
Time = fromordinal_vectorized(Time)

nse_calculator = SingleNSE()
df1 = pd.DataFrame()
df = pd.DataFrame()
df['STAID'] = dataset.gages['STAID']
df['KGE'] = seqKGE(y_pred, y_true, basin).cpu().numpy()
df['NSE'] = nse_calculator(y_pred, y_true, basin).cpu().numpy()
df1['basin'] = basin.cpu().numpy().ravel()
df1['y_true'] = y_true_denorm.ravel()
df1['y_pred'] = y_pred_denorm.ravel()
df1['time'] = Time.ravel()
df.to_csv('basin-results6.csv', index=None)
df1.to_csv('basin-data6.csv', index=None)

In [ ]:
# print(df['NSE'].mean())
# print(y_mean)

In [ ]:
# import hydra
# from hydra.utils import instantiate
# from trainer.loss import NMSELoss
# from trainer import Trainer
# import logging
# from hydra.experimental import compose, initialize

# def main(cfg):

#     logger = logging.getLogger('trainer')
#     # 定义里程碑，例如在第700个epoch时改变学习率
#     milestones = [700]
#     # 定义衰减比例
#     gamma = 0.5
#     learning_rate = 0.0001
#     n, accumulation = 0, 0
#     for train_loader, test_loader in datasplit:
#         # build model and print its architecture
#         model = BALSTM(input_size_sta, input_size_dyn, input_size_glo, hidden_size = 128,
#                        output_size = 1, num_layers = 1, drop_prob = 0.4)
#         trainable_params = filter(lambda p: p.requires_grad, model.parameters())
#         logger.info(model)
#         logger.info(f'Trainable parameters: {sum([p.numel() for p in trainable_params])}')

#         # get function handles of loss and metrics
#         criterion = NMSELoss(dataset)
#         metrics = [instantiate(met) for met in cfg.trainer['metrics']]
#         # build optimizer and learning rate scheduler
#         optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
#         lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)
#         # train and accumulate metric
#         trainer = Trainer(model, criterion, metrics, optimizer,
#                             config = cfg,
#                             data_loader=train_loader,
#                             valid_data_loader=test_loader,
#                             lr_scheduler=lr_scheduler,
#                             round=n)
#         accumulation += trainer.train()
#         n += 1
#     return accumulation / n
# if __name__ == "__main__":
#     initialize(config_path='configs')
#     cfg = compose(config_name='config')
#     main(cfg)